In [1]:
import os
HOME = os.getcwd()
print(HOME)

c:\Users\WilhelmB\Documents\DiagAssistAI


In [1]:
!python scripts/install_deps.py

Looking in indexes: https://download.pytorch.org/whl/cu121


In [2]:
!pip install -r requirements.txt

In [3]:
%pip install -e .  --verbose

Using pip 24.0 from c:\Users\WilhelmB\miniconda3\envs\prueba2\lib\site-packages\pip (python 3.10)
Obtaining file:///C:/Users/WilhelmB/Documents/DiagAssistAI
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'
  Building editable for LangSam (pyproject.toml): started
  Building editable for LangSam (pyproject.toml): finished with status 'done'
  Created wheel for LangSam: filename=langsam-0.1.0-py3-none-any.whl size=5437 sha256=672eeae9442a0b007213c1a3fab1d4177c7fa0bb2ac9507eea7e15d05fae5d9c
  Stored in directory: C:\Users\WilhelmB\AppData\Local\Temp\

  Running command pip subprocess to install build dependencies
    Using cached poetry_core-1.9.0-py3-none-any.whl.metadata (3.5 kB)
  Using cached poetry_core-1.9.0-py3-none-any.whl (309 kB)
  Running command Checking if build backend supports build_editable
  Running command Getting requirements to build editable
  Running command Preparing editable metadata (pyproject.toml)
  Running command Building editable for LangSam (pyproject.toml)


In [4]:
!python setup.py build_ext --inplace --verbose

Compiling with CUDA
running build_ext
building 'groundingdino._C' extension
creating build
creating build\temp.win-amd64-cpython-310
creating build\temp.win-amd64-cpython-310\Release
creating build\temp.win-amd64-cpython-310\Release\Users
creating build\temp.win-amd64-cpython-310\Release\Users\WilhelmB
creating build\temp.win-amd64-cpython-310\Release\Users\WilhelmB\Documents
creating build\temp.win-amd64-cpython-310\Release\Users\WilhelmB\Documents\DiagAssistAI
creating build\temp.win-amd64-cpython-310\Release\Users\WilhelmB\Documents\DiagAssistAI\src
creating build\temp.win-amd64-cpython-310\Release\Users\WilhelmB\Documents\DiagAssistAI\src\groundingdino
creating build\temp.win-amd64-cpython-310\Release\Users\WilhelmB\Documents\DiagAssistAI\src\groundingdino\models
creating build\temp.win-amd64-cpython-310\Release\Users\WilhelmB\Documents\DiagAssistAI\src\groundingdino\models\GroundingDINO
creating build\temp.win-amd64-cpython-310\Release\Users\WilhelmB\Documents\DiagAssistAI\src\gro

c:\Users\WilhelmB\miniconda3\envs\prueba2\lib\site-packages\torch\utils\cpp_extension.py:495: UserWarning: Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the slow distutils backend.
  warnings.warn(msg.format('we could not find ninja.'))


In [5]:
from groundingdino import _C

c:\Users\WilhelmB\miniconda3\envs\prueba2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
from LangSAM.SamLangDino import SamDino
from src.DataSets.Mamitas_Thermal_Dataset.Mamitas_Dataset import Mamitas_Thermal_Feet_Dataset, ToBoolTensor, PermuteTensor 
from torchvision.transforms import transforms
import torch

In [8]:
model = SamDino(SAM='vit_h')

final text_encoder_type: bert-base-uncased


In [4]:
transform_mask = transforms.Compose([
    ToBoolTensor(),
    PermuteTensor((1,2,0))
])
transform_img = transforms.Compose([
    transforms.ToTensor(),
    transforms.ConvertImageDtype(torch.uint8),
    PermuteTensor((1,2,0))
])

_, val_dataset = Mamitas_Thermal_Feet_Dataset().generate_dataset_with_val(torch_dataset=True,
                                                   batch_size=38,
                                                   shuffle=False,
                                                   split_val=0.2,
                                                   merge_image=True,
                                                   transform_mask=transform_mask,
                                                   transform_img=transform_img)


Dataset URL: https://www.kaggle.com/datasets/lucasiturriago/mamitas-thermal-feet
Train_dataset: 150
Val_dataset: 38


In [5]:
text_prompt="feet,foot"
box_threshold=0.10
text_threshold=0.25
process_box_threshold = 0.10

boxes,logits,phrases,images,ids,masks_o,masks_p = model.LangSam_batch(image_data=val_dataset,
                                                                      text_prompt=text_prompt,
                                                                      box_threshold=box_threshold,
                                                                      process_box_threshold=process_box_threshold,
                                                                      text_threshold=text_threshold,
                                                                      mode_predict='box',
                                                                      mode_data='batch')

NameError: name '_C' is not defined